**This code computes the decomposed/monolithic solution of a Closed-Loop DRTO problem and implement in the plant.**

- The model is the nonlinear version of the bioreactor of Case Study 2 from:*Gao, Ling. "Modeling and dynamics analyses of immobilized CSTR bioreactor using transfer function model." 2012 International Symposium on Information Technologies in Medicine and Education. Vol. 2. IEEE, 2012.*
- The underlying control structure (modeled in the CLDRTO model) contains one constrained MPC, which is represented by its KKT conditions in the DRTO problem. The complementarity constraints of the MPC are "moved" to the objective function as a  l1 penalty
- The problem is posed as a scenario-based stochastic programming problem
- The uncertainty represented by the scenarios comes from the the maximum reaction rate (Vm) 
- We run a Monte Carlo Analysis of the performance of both strategies for an arbitrary number of scenarios

--> see jupyter notebook "Preliminaries" and "Monolithic and Decomposed CLRTO Comparison" for details

AUTHOR: Jose Matias <assumpcj@mcmaster.ca>
DATE: September 2023

[1.Model and Simulation Parameters](#sec1) <br>
[2.Building Auxiliary Functions](#sec2)<br>
[2.1. MPC matrices builder](#subsec21)<br>
[2.2. MPC solver](#subsec22)<br>
[2.3. Monolithic CLDRTO](#subsec23)<br>
[2.4. Decomposed CLDRTO](#subsec24)<br>
[2.5. Plant Model](#subsec25)<br>
[3. Implementing CL-DRTO in closed loop with the plant](#sec3)<br>

In [1]:
using LinearAlgebra, JuMP, Ipopt, Plots, LaTeXStrings, CSV, DataFrames, DifferentialEquations, Distributions, Random, Statistics

<a id="sec1"></a>

# Model and Simulation Parameters

- Simulation Parameters

In [2]:
# Simulation sampling time
dt = 1.0 #[h]

# Integration step (plant)
tspan = (0.0,dt)

# Optimization horizon 
ndrto = 20 + (8 - 1) # p = 8 *controller horizon

# Optimization control horizon 
mD = 20 

# Optimization horizon
Δdrto = 4 # dt_DRTO

# integration (BE) sampling time
n_fe_dt = 5 # number of finite elements/simulation sampling time 

# finite element length
h = dt/n_fe_dt 

# number of finite elements 
nfe = ndrto*n_fe_dt

# Set seed value for choosing the plant parameter 
seed = 42
Random.seed!(seed)

# number of inputs
nu = 1 #[-]
# number of outputs
ny = 1 #[-]
# number of states
nx = 2 # [-]

# INITIAL CONDITION (from paper)
C0 = 0.358 #[g/L]
P0 = 0.642 # [g/L]

# States (algebraic and differential): C, P
x0 = [C0,P0]

# INPUTS (from paper)
Cin0 = 1 #[g/L]
u0 = Cin0;

- Model Parameters

In [3]:
# maximum reaction rate (NOTE CHANGE IN PARAMETERS FROM LINEAR)
Vm_nom = 0.75 # [1/h]

# reaction constant (inhibition parameter)
Ks = 0.5 # [g/L]

# Dilution rate ==> D = F/V, Fbar = 2 [L/h], V = 4 [L] 
D = 0.5; # [1/h] 

In [4]:
# Uncertain parameters: Vm 
# nominal value
pNom = 0.75
# standard deviation associated with uncertainty (\pm 20%)
deltaPar = 0.2*pNom; 

- Economic Optimization Problem Parameters

In [5]:
# feed concentration limitation  (2.0 vs. 5.0)
Cin_max = 5.0 #[g/L] 

# product setpoint limitation
Psp_max = 1.2 # [g/L]

# Soft upper bounds on P weight
pUB = 1.05 # [g/L]

# objective function weight for P penalization term 
wP = 100

# Objective function: target tracking
pTrack = 1.0 # [g/L]

# setting bounds (all arbitrary - not in deviation form!!) 
uMax = 5.0 # [g/L]
uMin = 0.0

yspMax = 1.2 # [g/L]
yspMin = 0.0;

- MPC Parameters

In [6]:
##########
# MODELS #
##########
# Linearization point
uref = 1.0
xref = [0.3660;0.6340]
yref = 0.6340

# 100 pre-computed models based on parametric uncertainty Vm (computed using matlab: BioReactorLinearization.m)
bioModels = CSV.read("C:\\Users\\MACC-Jose\\Documents\\GitHub\\BD\\CLDRTO Case Study 3 - Bioreactor (nonlinear)\\BioreactorModel", DataFrame; header=false);
#print(bioModels)

# extracting the matrices from the model array computed previously
nNom = 50 # nominal model --> index = 50

A = Matrix(bioModels[2*(nNom - 1) + 1:2*(nNom - 1) + 2,1:2]) 
B = bioModels[2*(nNom - 1) + 1:2*(nNom - 1) + 2,3]
C = [0 1];

# initial value of xMPC
xMPC0 = x0 - xref
de0 = 0

##########
# TUNING #
##########
# prediction horizon 
p = 8

# control horizon 
m = 2

# output tracking weight
q = 1

# move supression weight
r = 1

# input bounds (not in deviation form)
u_mpc_lb = 0.0
u_mpc_ub = Cin_max

##################
# KKT CONDITIONS #
##################
# matrix to compute the gradients of the input bound constraints
conMatrix = [-I(m); I(m)]

# regularization term OF for complementarities (1e1)
rho_cc = 1e1;

- Preparing Orthogonal Collocation

In [7]:
# Parameters for orthogonal collocation
#  Radau Collocation Points 1st degree polynomial == Backward Euler
radau  = [0.0 1.00000]

# number of collocation points (initial point is also considered)
ncp = length(radau) 

# Collocation parameters and radau time series
colmat = [-1.0 -1.0;
          1.0 1.0];          

- Bender's Decomposition

In [8]:
# max number of iteration (50)
benIterMax = 150

# optimality gap (relative) tolerance (1e-2)
optGap = 1e-3

# parameter that weights the nominal trajectory tracking term (1e-2)
η_n = 0.0;

<a id="sec2"></a>

# Building auxiliary functions

<a id="subsec21"></a>

## MPC matrices builder

In [9]:
function MPC_matrices(A,B,C,p,m,q,r)
    ###########
    # inputs: #
    ###########
    # A,B,C - System model
    # p, m - prediction and control horizon
    # q, r - setpoint and input usage tuning parameters
    
    # extracting dimensions from system model
    nx = size(A,1) # states
    nu = size(B,2) # inputs
    ny = size(C,1) # outputs
    
    # adding disturbance model
    Ad = [A zeros(nx,ny); zeros(ny,nx) I(ny)]
    Bd = [B; zeros(ny,nu)]
    Cd = [C ones(ny,ny)]
    
    # building matrices for MPC
    Psi = Cd*Ad
    for ii in 2:p
        Psi = [Psi;  Cd*Ad^ii]
    end

    # Computing Dynamic Matrix
    a = Cd*Bd
    for ii in 2:p
        a = [a; Cd*Ad^(ii - 1)*Bd]
    end
    DynM = a

    for ii in 1:(m - 2)
        a = [zeros(ny,nu);a[1:(p-1)*ny,:]]
        DynM = [DynM  a]
    end

    # adjusting dynamic matrix for since p > m (last column)
    b = Cd*Bd

    Ai = I(nx+1) # adding disturbance to the states
    for ii = 1:(p - m)
        Ai = Ai + Ad^ii
        b = [b;Cd*Ai*Bd]
    end

    Theta=[DynM [zeros(ny*(m-1),nu);b]]
    
    # Creating Qbar and Rbar matrices
    Qbar = Diagonal([q for ii in 1:p])
    Rbar = Diagonal([r for ii in 1:m])

    # Creating input movement OF penalty matrix 
    M=[zeros((m-1)*nu,nu) I(nu*(m-1)); zeros(nu) zeros(nu,nu*(m-1))]
    Ibar=[I(nu); zeros(nu*(m-1),nu)]
    IM = I(nu*m) - M';
    
    # Matrix H
    H = Theta'*Qbar*Theta + IM'*Rbar*IM;
    
    return Dict('A' => A, 'B' => B, 'C' => C, 'H' => H, 'Θ' => Theta, 'Ψ' => Psi, 'Q' => Qbar, 'I' => Ibar, 'R' => Rbar, 'M' => IM)
end;

<a id="subsec23"></a>

## MPC solver

- Building function for computing MPC solution (solving QP)

In [10]:
function solveMPC(xkk,dekk,uk_1,yspk,umin,umax,yref,uref,mpc,p,m)
    ###########
    # inputs: #
    ###########
    # xkk - current states (MPC model) 
    # dekk - disturbance estimate 
    # uk_1 - inputs at the current iteration, already implemented on the plant
    # yspk - setpoint trajectory computed by CL-DRTO
    # umin, umax - input bounds
    # yref, uref - reference points for computing deviation variables
    # mpcModel - System model
    # p, m - prediction and control horizon
    
    MPC_con = Model(Ipopt.Optimizer)
    set_silent(MPC_con)
    
    ####################
    # Set up variables #
    ####################
    # inputs computed by MPCs
    @variable(MPC_con, (umin - uref) ≤ u[1:m] ≤ (umax - uref))
    
    ######################
    # Set up expressions #
    ######################
    @expression(MPC_con, cfT, (mpc['Ψ']*[xkk;dekk] - (yspk .- yref))'*mpc['Q']*mpc['Θ'] - (uk_1 - uref)'*mpc['I']'*mpc['R']*mpc['M'])
    
    
    # Option 1. solving as a constrained MPC
    @objective(MPC_con, Min, sum(u[jj]*sum(mpc['H'][jj,ii]*u[ii] for ii in 1:m) for jj in 1:m) + 2*sum(cfT[jj]*u[jj] for jj = 1:m))

    # Option 2. solving exactly like in the CLDRTO implementation (for debugging)
    #@constraint(MPC_con, MPCsol[jj = 1:m], sum(mpc['H'][jj,ii]*u[ii] for ii in 1:m) + cfT[jj] == 0.0)    
    #@objective(MPC_con, Min, -1)

    #################
    # Solve Problem #
    #################
    optimize!(MPC_con)
    
    if has_values(MPC_con)
        flag = 1
    else
        flag = 0
    end

    #calling values of the solved problem
    ϕ = objective_value(MPC_con)
    uArray = value.(u) .+ uref
    return Dict('ϕ' => ϕ,'u' => uArray, 'f' => flag)
end;

<a id="subsec23"></a>

## Monolithic solution of scenario-based CLDRTO

In [11]:
function MS_CLDRTO(C_0,P_0,u_0,xMPC_0,de_0,VmTemp,ns,solNom)
    ###########
    # inputs: #
    ###########
    # C_0, P_0 - substrate and biomass concentration at the current iteration (beginning of DRTO horizon)
    # u_0 - current input implemented in the plant
    # xMPC_0 - current MPC states
    # de_0 - current plant/MPC model deviation
    # VmTemp - vector containing all the values of the uncertain paramete
    # nScen - number of scenarios
    # solNom - nominal solution
    
    # Assuming equiprobable scenarios
    pScen = 1.0/ns
    
    # JuMP model
    model_ms = Model(Ipopt.Optimizer)
    set_optimizer_attribute(model_ms, "warm_start_init_point", "yes")
    #set_optimizer_attribute(model_ms, "print_level", 5)
    set_optimizer_attribute(model_ms, "max_cpu_time", 300.0) # 5 min
    set_silent(model_ms)

    ####################
    # Set up variables #
    ####################
     # DRTO model variables
    @variable(model_ms, C[1:nfe, 1:ncp, 1:ns] ≥ 0.0)    # [g/L] substrate concentration (diff states)
    @variable(model_ms, P[1:nfe, 1:ncp, 1:ns] ≥ 0.0)    # [g/L] biomass concentration (diff states)
    @variable(model_ms, Cdot[1:nfe, 1:ncp, 1:ns])       # [g/L/h] substrate concentration (time derivative)
    @variable(model_ms, Pdot[1:nfe, 1:ncp, 1:ns])       # [g/L/h] biomass concentration (time derivative)
    @variable(model_ms, delta_p[1:nfe, 1:ns] ≥ 0.0)     # [g/L] soft constraint on P
    @variable(model_ms, 0.0 ≤ Cin[1:ndrto, 1:ns] ≤ Cin_max) # [g/L] substrate feed to reactor

    # MPC model variables (SISO) -> deviation form
    @variable(model_ms, xMPC[1:nx,1:ndrto, 1:ns])
    @variable(model_ms, uMPC[1:ndrto,1:m, 1:ns]) 

    # MPC <-> DRTO model deviation
    @variable(model_ms, de[1:(ndrto - 1), 1:ns])

    # Constrained MPC KKT conditions (upper and lower bounds for each input)
    @variable(model_ms, mu_g[1:mD,1:(2*m), 1:ns] ≥ 0, start = 0) 

    # setpoints for the controllers sent to the plant (*CL-DRTO degrees of freedom)
    @variable(model_ms, yspMin ≤ ysp[1:ndrto, 1:ns] ≤ yspMax)

    
    # non-anticipativity constraints
    @variable(model_ms, yspMin ≤ yspNA[1:Δdrto] ≤ yspMax)
    
    #####################################
    # Set up initial guesses for solver #
    #####################################
    if solNom isa Dict # if gives flexibility to NOT initialize variables
        for ss in 1:ns
            for kk in 1:nfe
                for jj in 1:ncp
                    set_start_value(C[kk,jj,ss], solNom['C'][kk,jj,1])
                    set_start_value(P[kk,jj,ss], solNom['P'][kk,jj,1])
                end # collocation points loop
            end # end finite elements loop
            
            for kk in 1:ndrto
                # MV's
                set_start_value(Cin[kk,ss], solNom['I'][kk,1])
                
                for mm in 1:m
                    set_start_value(uMPC[kk,mm,ss], solNom['I'][kk,1] - uref)
                end
            end
            
            for kk in 1:mD
                for mm in 1:(2*m)
                    set_start_value(mu_g[kk,mm,ss], solNom['μ'][kk,mm,1])
                end
            end # drto sampling time loop
        end # scenarios loop
    else
        # Set up initial guesses for solver: using opt. steady-state previoulsy computed
        for ss in 1:ns
            for kk in 1:nfe
                for jj in 1:ncp
                    set_start_value(C[kk,jj,ss], C0) 
                    set_start_value(P[kk,jj,ss], P0)
                end # collocation points loop
            end # finite elements loop

            for kk in 1:ndrto
                # MV's
                set_start_value(Cin[kk,ss], Cin0)

                for mm in 1:m
                    set_start_value(uMPC[kk,mm,ss], Cin0 - uref)
                end
            end 
        end # scenarios loop
    end
    
    ##############
    # DRTO model #
    ##############
    # [g/L/h] Immobilized enzyme or cell catalytic reaction rate (defined by Michaelis-Menten equation)
    @NLexpression(model_ms, rc[kk = 1:nfe, jj = 1:ncp, ss = 1:ns], Vm_nom*C[kk,jj,ss]/(Ks + C[kk,jj,ss]))

    @NLconstraints(model_ms, begin
        #########
        # MODEL #
        #########    
        # reactor
        r_C[kk = 1:nfe, jj = 1:ncp, ss = 1:ns], Cdot[kk,jj,ss] == D*(Cin[cld(kk,n_fe_dt),ss] - C[kk,jj,ss]) - rc[kk,jj,ss] 
        r_P[kk = 1:nfe, jj = 1:ncp, ss = 1:ns], Pdot[kk,jj,ss] == rc[kk,jj,ss] - D*P[kk,jj,ss]
        #########################    
        # Collocation Equations #    
        #########################
        # initial point
        coll_C_0[ss = 1:ns], C[1,1,ss] == C_0 
        coll_P_0[ss = 1:ns], P[1,1,ss] == P_0  
        # set up collocation equations 
        coll_C_n[kk = 1:nfe, jj = 2:ncp, ss = 1:ns], sum(colmat[ii,jj]*C[kk,ii,ss] for ii in 1:ncp) - h*Cdot[kk,jj,ss] == 0 
        coll_P_n[kk = 1:nfe, jj = 2:ncp, ss = 1:ns], sum(colmat[ii,jj]*P[kk,ii,ss] for ii in 1:ncp) - h*Pdot[kk,jj,ss] == 0
        # continuinity equations
        coll_C_con[kk = 2:nfe, ss = 1:ns], C[kk,1,ss] - C[kk - 1,ncp,ss] == 0 
        coll_P_con[kk = 2:nfe, ss = 1:ns], P[kk,1,ss] - P[kk - 1,ncp,ss] == 0 
    end)

    ##########################
    # MPC model and solution #
    ##########################
    # N.B. inside MPC everything is in deviation variables
    @expressions(model_ms, begin
        cfT_1[ss = 1:ns], (mpc['Ψ']*[xMPC_0;de_0] - (ysp[1:p,ss] .- yref))'*mpc['Q']*mpc['Θ'] - (u0 - uref)*mpc['I']'*mpc['R']*mpc['M']
        cfT[kk = 1:(mD - 1),ss = 1:ns], (mpc['Ψ']*[xMPC[:,kk,ss];de[kk,ss]] - (ysp[(kk + 1):(kk + 1 + p - 1),ss] .- yref))'*mpc['Q']*mpc['Θ'] - uMPC[kk,1,ss]*mpc['I']'*mpc['R']*mpc['M']
        # Input bounds
        MPC_u_upper[kk = 1:mD,uu = 1:m,ss = 1:ns], (uMax - uref) - uMPC[kk,uu,ss]
        MPC_u_lower[kk = 1:mD,uu = 1:m,ss = 1:ns], uMPC[kk,uu,ss] - (uMin - uref)
    end)

    @constraints(model_ms, begin
        #############
        # MPC model #
        #############
        MPC_dyn_model_1[ss = 1:ns], xMPC[:,1,ss] .== mpc['A']*xMPC_0 + mpc['B']*(u0 - uref)
        MPC_dyn_model[kk = 1:(ndrto - 1),ss = 1:ns], xMPC[:,kk + 1,ss] .== mpc['A']*xMPC[:,kk,ss] + mpc['B']*uMPC[kk,1,ss]
        MPC_model_dev[kk = 1:(ndrto - 1),ss = 1:ns], de[kk,ss] == (P[kk,ncp,ss] - yref) - xMPC[2,kk,ss]
        ################
        # MPC solution #
        ################
        # stationarity
        MPC_sol_sta_1[ss = 1:ns], uMPC[1,:,ss]'*mpc['H'] + cfT_1[ss] +  mu_g[1,:,ss]'*conMatrix .== 0
        MPC_sol_sta[kk = 1:(mD - 1), ss = 1:ns], uMPC[kk + 1,:,ss]'*mpc['H'] + cfT[kk,ss] +  mu_g[kk + 1,:,ss]'*conMatrix .== 0
        # primal feasibility
        MPC_sol_feas_ub[kk = 1:mD,uu = 1:m,ss = 1:ns], MPC_u_upper[kk,uu,ss] ≥ 0
        MPC_sol_feas_lb[kk = 1:mD,uu = 1:m,ss = 1:ns], MPC_u_lower[kk,uu,ss] ≥ 0
    end)
    
    @constraints(model_ms, begin   
        ###############
        # CONSTRAINTS #
        ###############
        # ! Enforcing only at the end of the finite element
        # P soft constraints
        soft_P[kk = 1:nfe,ss = 1:ns], P[kk,ncp,ss] - delta_p[kk,ss] ≤ pUB
        # fixing setpoint changes after mD
        control_horizon[kk = (mD+1):ndrto,uu = 1:m,ss = 1:ns], uMPC[kk,uu,ss] == uMPC[mD,uu,ss]
        ############################
        # OPTIMIZATION CONSTRAINTS #
        ############################
        # connecting the MPC solution to the DRTO model (only the first element of the MV* is implemented)
        hc[kk = 1:ndrto,ss = 1:ns], (uMPC[kk,1,ss] + uref) - Cin[kk,ss] == 0.0
        ##################################
        # NON-ANTICIPATIVITY CONSTRAINTS #
        ##################################
        nonAnt[kk = 1:Δdrto,ss = 1:ns], ysp[kk,ss] - yspNA[kk] == 0.0
    end)
    
    # Set up objective function     
    if ns > 1    
        @NLobjective(model_ms, Min, pScen*sum(sum((P[kk,ncp,ss] - pTrack)^2 + wP*delta_p[kk,ss]^2 for kk in 1:nfe)
        + rho_cc*sum(mu_g[kk,uu,ss]*MPC_u_upper[kk,uu,ss] + mu_g[kk,m + uu,ss]*MPC_u_lower[kk,uu,ss] for uu = 1:m, kk = 1:mD) 
        for ss in 1:ns) +  η_n*sum((yspNA[kk] - solNom['s'][kk,1])^2 for kk in 1:Δdrto))
    else
        @NLobjective(model_ms, Min, pScen*sum(sum((P[kk,ncp,ss] - pTrack)^2 + wP*delta_p[kk,ss]^2 for kk in 1:nfe)
        + rho_cc*sum(mu_g[kk,uu,ss]*MPC_u_upper[kk,uu,ss] + mu_g[kk,m + uu,ss]*MPC_u_lower[kk,uu,ss] for uu = 1:m, kk = 1:mD) 
        for ss in 1:ns))
    end
    
    #################
    # Solve Problem #
    #################
    optimize!(model_ms)
    
    status = termination_status(model_ms)
    #display(status)
    
    if status == MOI.OPTIMAL || status == MOI.ALMOST_OPTIMAL || status == MOI.LOCALLY_SOLVED
        # Proved optimality or Optimal within relaxed tolerances
        # solution time
        timeSol = solve_time(model_ms)

        flag = 1
        # #primal_status(m)

        #calling values of the solved problem
        ϕ = objective_value(model_ms)
        yspArray = value.(ysp)
        CArray = value.(C)
        PArray = value.(P)
        CinArray = value.(Cin)
        
        muArray = value.(mu_g)
        
        outputFun = Dict('ϕ' => ϕ,
                        't' => timeSol,
                        'f' => flag, 
                        's' => yspArray,
                        'I' => CinArray,
                        'C' => CArray,
                        'P' => PArray,
                        'μ' => muArray)
        
        return outputFun
    else
        # Handle other cases.
        outputFun = Dict('f' => 0,
                         't' => NaN,
                         'ϕ' => NaN)
                        
        
        return outputFun
    end
end;

<a id="subsec24"></a>

## Decomposed CLDRTO

- Modeling Subproblem

In [12]:
function subp(yspFixed,C_0,P_0,u_0,xMPC_0,de_0,VmScen,pScen,solNom)    
    ###########
    # inputs: #
    ###########
    # yspFixed - setpoints determined by the master problem 
    # C_0, P_0 - substrate and biomass concentration at the current iteration (beginning of DRTO horizon)
    # u_0 - current input implemented in the plant
    # xMPC_0 - current MPC states
    # de_0 - current plant/MPC model deviation
    # VmScen - vector containing the values of the uncertain parameter for a given scenario
    # pScen - probability of a given scenario
    # solNom - nominal solution
    
    # JuMP model
    model_sub = Model(Ipopt.Optimizer)
    set_optimizer_attribute(model_sub, "warm_start_init_point", "yes")
    #set_optimizer_attribute(model_sub, "print_level", 5)
    set_optimizer_attribute(model_sub, "max_cpu_time", 300.0) # 5 min
    set_silent(model_sub)
    
     ####################
    # Set up variables #
    ####################
     # DRTO model variables
    @variable(model_sub, C[1:nfe, 1:ncp] ≥ 0.0)    # [g/L] substrate concentration (diff states)
    @variable(model_sub, P[1:nfe, 1:ncp] ≥ 0.0)    # [g/L] biomass concentration (diff states)
    @variable(model_sub, Cdot[1:nfe, 1:ncp])       # [g/L/h] substrate concentration (time derivative)
    @variable(model_sub, Pdot[1:nfe, 1:ncp])       # [g/L/h] biomass concentration (time derivative)
    @variable(model_sub, delta_p[1:nfe] ≥ 0.0)     # [g/L] soft constraint on P
    @variable(model_sub, 0.0 ≤ Cin[1:ndrto] ≤ Cin_max) # [g/L] substrate feed to reactor

    # MPC model variables (SISO) -> deviation form
    @variable(model_sub, xMPC[1:nx,1:ndrto])
    @variable(model_sub, uMPC[1:ndrto,1:m]) 

    # MPC <-> DRTO model deviation
    @variable(model_sub, de[1:(ndrto - 1)])

    # Constrained MPC KKT conditions (upper and lower bounds for each input)
    @variable(model_sub, mu_g[1:mD,1:(2*m)] ≥ 0, start = 0) 

    # setpoints for the controllers sent to the plant (*CL-DRTO degrees of freedom)
    @variable(model_sub, yspMin ≤ ysp[1:ndrto] ≤ yspMax)
    
    #####################################
    # Set up initial guesses for solver #
    #####################################
    if solNom isa Dict # if gives flexibility to NOT initialize variables
        for kk in 1:nfe
            for jj in 1:ncp
                set_start_value(C[kk,jj], solNom['C'][kk,jj,1])
                set_start_value(P[kk,jj], solNom['P'][kk,jj,1])
            end # collocation points loop
        end # end finite elements loop

        for kk in 1:ndrto
            # MV's
            set_start_value(Cin[kk], solNom['I'][kk,1])

            for mm in 1:m
                set_start_value(uMPC[kk,mm], solNom['I'][kk,1] - uref)
            end
        end
            
        for kk in 1:mD
            for mm in 1:(2*m)
                set_start_value(mu_g[kk,mm], solNom['μ'][kk,mm,1])
            end
        end # drto sampling time loop
    else
    # Set up initial guesses for solver: using opt. steady-state previoulsy computed
        for kk in 1:nfe
            for jj in 1:ncp
                set_start_value(C[kk,jj], C0) 
                set_start_value(P[kk,jj], P0)
            end # collocation points loop
        end # finite elements loop

        for kk in 1:ndrto
            # MV's
            set_start_value(Cin[kk], Cin0)

            for mm in 1:m
                set_start_value(uMPC[kk,mm], Cin0 - uref)
            end
        end 
    end
    
    ##############
    # DRTO model #
    ##############
    # [g/L/h] Immobilized enzyme or cell catalytic reaction rate (defined by Michaelis-Menten equation)
    @NLexpression(model_sub, rc[kk = 1:nfe, jj = 1:ncp], Vm_nom*C[kk,jj]/(Ks + C[kk,jj]))

    @NLconstraints(model_sub, begin
        #########
        # MODEL #
        #########    
        # reactor
        r_C[kk = 1:nfe, jj = 1:ncp], Cdot[kk,jj] == D*(Cin[cld(kk,n_fe_dt)] - C[kk,jj]) - rc[kk,jj] 
        r_P[kk = 1:nfe, jj = 1:ncp], Pdot[kk,jj] == rc[kk,jj] - D*P[kk,jj]
        #########################    
        # Collocation Equations #    
        #########################
        # initial point
        coll_C_0, C[1,1] == C_0 
        coll_P_0, P[1,1] == P_0  
        # set up collocation equations 
        coll_C_n[kk = 1:nfe, jj = 2:ncp], sum(colmat[ii,jj]*C[kk,ii] for ii in 1:ncp) - h*Cdot[kk,jj] == 0 
        coll_P_n[kk = 1:nfe, jj = 2:ncp], sum(colmat[ii,jj]*P[kk,ii] for ii in 1:ncp) - h*Pdot[kk,jj] == 0
        # continuinity equations
        coll_C_con[kk = 2:nfe], C[kk,1] - C[kk - 1,ncp] == 0 
        coll_P_con[kk = 2:nfe], P[kk,1] - P[kk - 1,ncp] == 0 
    end)

    ##########################
    # MPC model and solution #
    ##########################
    # N.B. inside MPC everything is in deviation variables
    @expressions(model_sub, begin
        cfT_1, (mpc['Ψ']*[xMPC_0;de_0] - (ysp[1:p] .- yref))'*mpc['Q']*mpc['Θ'] - (u0 - uref)*mpc['I']'*mpc['R']*mpc['M']
        cfT[kk = 1:(mD - 1)], (mpc['Ψ']*[xMPC[:,kk];de[kk]] - (ysp[(kk + 1):(kk + 1 + p - 1)] .- yref))'*mpc['Q']*mpc['Θ'] - uMPC[kk,1]*mpc['I']'*mpc['R']*mpc['M']
        # Input bounds
        MPC_u_upper[kk = 1:mD,uu = 1:m], (uMax - uref) - uMPC[kk,uu]
        MPC_u_lower[kk = 1:mD,uu = 1:m], uMPC[kk,uu] - (uMin - uref)
    end)

    @constraints(model_sub, begin
        #############
        # MPC model #
        #############
        MPC_dyn_model_1, xMPC[:,1] .== mpc['A']*xMPC_0 + mpc['B']*(u0 - uref)
        MPC_dyn_model[kk = 1:(ndrto - 1)], xMPC[:,kk + 1] .== mpc['A']*xMPC[:,kk] + mpc['B']*uMPC[kk,1]
        MPC_model_dev[kk = 1:(ndrto - 1)], de[kk] == (P[kk,ncp] - yref) - xMPC[2,kk]
        ################
        # MPC solution #
        ################
        # stationarity
        MPC_sol_sta_1, uMPC[1,:]'*mpc['H'] + cfT_1 +  mu_g[1,:]'*conMatrix .== 0
        MPC_sol_sta[kk = 1:(mD - 1)], uMPC[kk + 1,:]'*mpc['H'] + cfT[kk] +  mu_g[kk + 1,:]'*conMatrix .== 0
        # primal feasibility
        MPC_sol_feas_ub[kk = 1:mD,uu = 1:m], MPC_u_upper[kk,uu] ≥ 0
        MPC_sol_feas_lb[kk = 1:mD,uu = 1:m], MPC_u_lower[kk,uu] ≥ 0
    end)
    
    @constraints(model_sub, begin   
        ###############
        # CONSTRAINTS #
        ###############
        # ! Enforcing only at the end of the finite element
        # P soft constraints
        soft_P[kk = 1:nfe], P[kk,ncp] - delta_p[kk] ≤ pUB
        # fixing setpoint changes after mD
        control_horizon[kk = (mD+1):ndrto,uu = 1:m], uMPC[kk,uu] == uMPC[mD,uu]
        ############################
        # OPTIMIZATION CONSTRAINTS #
        ############################
        # connecting the MPC solution to the DRTO model (only the first element of the MV* is implemented)
        hc[kk = 1:ndrto], (uMPC[kk,1] + uref) - Cin[kk] == 0.0
        ##################################
        # NON-ANTICIPATIVITY CONSTRAINTS #
        ##################################
        nonAnt[kk = 1:Δdrto], ysp[kk] - yspFixed[kk] == 0.0
    end)
    
    # Set up objective function      
    @NLobjective(model_sub, Min, pScen*(sum((P[kk,ncp] - pTrack)^2 + wP*delta_p[kk]^2 for kk in 1:nfe)
    + rho_cc*sum(mu_g[kk,uu]*MPC_u_upper[kk,uu] + mu_g[kk,m + uu]*MPC_u_lower[kk,uu] for uu = 1:m, kk = 1:mD)))
    
    #################
    # Solve Problem #
    #################    
    optimize!(model_sub)
    
    # solution time
    timeSol = solve_time(model_sub)
    
    status = termination_status(model_sub)
    #display(status)
    
    # checking if there is a solution to the problem
    if status == MOI.ALMOST_LOCALLY_SOLVED || status == MOI.LOCALLY_SOLVED
        # Proved optimality or Optimal within relaxed tolerances
        # we add an optimality cut
        flag = 1
        # #primal_status(m)

        #calling values of the solved problem
        ϕ = objective_value(model_sub)
        yspArray = value.(ysp)
        CArray = value.(C)
        PArray = value.(P)
        CinArray = value.(Cin)
        
        muArray = value.(mu_g)
        
        # lagrange multipliers associated with the nonantecipativity constraints
        λ = dual.(nonAnt)
        
        outputDict = Dict('ϕ' => ϕ,
                        't' => timeSol,
                        'f' => flag, 
                        'a' => status,
                        's' => yspArray,
                        'I' => CinArray,
                        'C' => CArray,
                        'P' => PArray, 
                        'μ' => muArray,
                        'λ' => λ)
        
        return outputDict
    else
        #i.e. no feasible solution --> Add feasibility cut
        #display("Feasibility cut: scenario $(scen)")        
        
        # ! soft bounds on ysp
        delete_lower_bound.(ysp[1:pD])
        delete_upper_bound.(ysp[1:pD])
           
        # ! soft bounds on u
        delete(model_sub, MPC_sol_feas_ub)
        delete(model_sub, MPC_sol_feas_lb)
        unregister(model_sub, :MPC_sol_feas_ub)
        unregister(model_sub, :MPC_sol_feas_lb)
        
        ####################
        # Set up variables #
        ####################    
        # slack variables
        @variable(model_sub, delta_ysp_u[1:ndrto] ≥ 0.0)
        @variable(model_sub, delta_ysp_l[1:ndrto] ≥ 0.0)
        @variable(model_sub, delta_u_u[1:mD,1:m] ≥ 0.0)
        @variable(model_sub, delta_u_l[1:mD,1:m] ≥ 0.0)
        
        ###########################
        # Always-feasible problem #
        ###########################        
        @constraint(model_sub, soft_ysp_upper[kk = 1:ndrto], ysp[kk] - yspMax ≤ delta_ysp_u[kk])
        @constraint(model_sub, soft_ysp_lower[kk = 1:ndrto], yspMin - ysp[kk] ≤ delta_ysp_l[kk])
        @constraint(model_sub, soft_u_upper[kk=1:mD,uu=1:m], u[kk,uu] - (uMax - u0) ≤ delta_u_u[kk,uu])
        @constraint(model_sub, soft_u_lower[kk=1:mD,uu=1:m], (uMin - u0) - u[kk,uu] ≤ delta_u_l[kk,uu])
        
       #############################
        # Set up objective function #
        #############################
        # Modifying the objective --> minimizing constraint violation
        @objective(model_sub, Min, sum(delta_p[kk] for kk in 1:nfe) 
            + sum(delta_ysp_u[kk] + delta_ysp_l[kk] for kk in 1:ndrto)
            + sum(delta_u_u[kk,uu] + delta_u_l[kk,uu] for uu in 1:m, kk in 1:mD))
      

        #@show sub
        # re-optimizing (now using the always feasible problem)
        optimize!(model_sub)
        
        # objective function of the feasibility problem
        o = objective_value(model_sub)

        # solution time (adding time to solve the always feasible subproblem)
        timeSol = solve_time(model_sub)
        
        # checking status of the solution
        status_sub = termination_status(model_sub)
        
        # flag for calling feasibility cuts (not optimality cuts)
        flag = 0
        
        #calling values of the solved problem
        yspArray = value.(ysp)
        CinArray = value.(Cin)
        CArray = value.(C)
        PArray = value.(P)
        muArray = value.(mu_g)
        
        #slacks
#         dysp_ub = value.(delta_ysp_u)
#         dysp_lb = value.(delta_ysp_l)
#         temp1 = sum(dysp_ub[kk] + dysp_lb[kk] for kk in 1:ndrto)
#         display("DB: SP slacks sum: $(temp1)")
        
#         dc_ub = value.(delta_u_u)
#         dc_lb = value.(delta_u_l)
#         temp2 = sum(sum(dc_ub[kk,uu] + dc_lb[kk,uu] for uu in 1:m) for kk in 1:mD)
#         display("DB: MPC input slacks sum: $(temp2)")
        
        # lagrange multipliers associated with the nonantecipativity constraints
        λ = dual.(nonAnt)
        
        outputDict = Dict('ϕ' => o,
                        't' => timeSol,
                        'f' => flag, 
                        'a' => status_sub,
                        's' => yspArray,
                        'I' => CinArray,
                        'C' => CArray,
                        'P' => PArray,
                        'μ' => muArray,
                        'λ' => λ)
        
        return outputDict
    end
end;

- Modeling Master problem

In [13]:
## Modeling Benders Decompostion
function BENS_CLDRTO(C_0,P_0,u_0,xMPC_0,de_0,VmTemp,ns,solNom)
    ###########
    # inputs: #(xInit,uInit,nReac,nScen,pScen,solNom,option)
    ###########
    # C_0, P_0 - substrate and biomass concentration at the current iteration (beginning of DRTO horizon)
    # u_0 - current input implemented in the plant
    # xMPC_0 - current MPC states
    # de_0 - current plant/MPC model deviation
    # VmTemp - vector containing all the values of the uncertain parameter (Vm)
    # ns - number of scenarios
    # solNom - nominal solution
    
    # saving opt gap array
    optGapArray = Vector{Float64}()
    
    # Assuming equiprobable scenarios
    pScen = 1.0/ns
    
    # flag for checking if last cut was opt. (1) or feas. (0)
    flagO = 1
    
    # timing solution via Benders
    timeSolBen = 0 
    # timing solution via parallel Benders
    timeSolParBen = 0 
    
    # Define master problem model
    master = Model(Ipopt.Optimizer)
    set_silent(master)
    
    #lower bound on scenario objective function approximation 
    alpha_down = 0.0; 
    
    # defining temp solution to avoid loop's scope
     ysp_nonⁱ = Vector{Float64}(undef,Δdrto)
    
    ####################
    # Set up variables #
    ####################
    # setpoints for the controllers sent to the plant
    @variable(master, yspMin ≤ ysp[1:Δdrto] ≤ yspMax)
    @variable(master, α ≥ alpha_down)
    
    ################################################
    # Solving problem with benders' decomposition  #
    ################################################
    for ii in 1:benIterMax
        
        ########################
        # Set up initial guess #
        ########################
        # check size of the optimality gap array
        s_opt_array = size(optGapArray)
        
        # always use nominal solution in the first iteration
        if ii == 1 
            for kk in 1:Δdrto
                set_start_value(ysp[kk], solNom['s'][kk,1])
            end 
        # check if MASTER problem got stuck in a solution, in this case use random guess   
        elseif s_opt_array[1] > 1 && abs(optGapArray[end] - optGapArray[end - 1]) < 0.001
            tempGuess = rand(Uniform(yspMin,yspMax),Δdrto)
            display("DB: random")

            for kk in 1:Δdrto
                set_start_value(ysp[kk], tempGuess[kk])
            end  
        # otherwise use previous solution
        else    
            for kk in 1:Δdrto
                set_start_value(ysp[kk],  ysp_nonⁱ[kk])
            end 
        end
          
        #############################
        # Set up objective function #
        #############################
        # "nudge" next solution towards nominal solution (working = 1e-4)
        @objective(master, Min, α + η_n*sum((ysp[kk] - solNom['s'][kk,1])^2 for kk in 1:Δdrto))
        
        # solve master problem (with Benders' cut)
        optimize!(master)

        # solution time
        timeSolMaster = solve_time(master)
        # extracting solution for the non-anticipativity constraints
        ysp_nonⁱ = value.(ysp) 
        
        # Master problem solution: lower bound of the original problem
        lb = value(α)
        
        timeSolBen = timeSolBen + timeSolMaster
        timeSolParBen = timeSolParBen + timeSolMaster # master cannot be paralellized 

        #######################
        # Solving subproblems #
        #######################
        # objective function
        o_temp = Vector{Float64}(undef,ns)
        # flag optimality / feasibility
        flag_temp = Vector{Float64}(undef,ns)
        # solution time
        solTime_temp = Vector{Float64}(undef,ns)
        # multipliers
        lambdaN_temp = Matrix{Float64}(undef,Δdrto,ns)
        
        for ss in 1:ns
            dicTemp = subp(ysp_nonⁱ,C_0,P_0,u_0,xMPC_0,de_0,VmTemp[ss],pScen,solNom)

            # saving files
            # objective function
            o_temp[ss] = dicTemp['ϕ']
            # solution time
            solTime_temp[ss] = dicTemp['t']
            # optimality / feasibility flag
            flag_temp[ss] = dicTemp['f']
            # multiplier
            for kk in 1:Δdrto
                lambdaN_temp[kk,ss] = dicTemp['λ'][kk]
            end
        end
        
        ##########################################################
        # If all subproblems were feasible, check optimality gap #
        ##########################################################
        if sum(flag_temp) == ns 
            # Subproblem solution: upper bound of the original problem
            ub = sum(o_temp)

            # solve subproblems to find the upper bound of the original problem solution 
            timeSolBen = timeSolBen + sum(solTime_temp)

            # assuming that subproblem solution is parallelized
            timeSolParBen = timeSolParBen + maximum(solTime_temp)   
            
            # compute optimality gap
            gap = abs(ub - lb)/abs(ub)
            display("DB: opt. gap $(gap)")
            push!(optGapArray, gap)
            
            # check if optimality gap is small enough
            # also, make sure last cut was an optimality cut due to change in OF 
            if gap < optGap && flagO == 1
                global yspSol_dec = ysp_nonⁱ
                global objFun_dec = ub
                global optGap_dec = gap
                global iter_dec = ii
                global time_dec = timeSolBen
                global time_max_dec = timeSolParBen
                break
            else # gap ≥ optGap 
                ##############################
                # --> adding optimality cuts #
                ##############################
                display(">>>>>> iteration $(ii):O")
                benderscutO = @constraint(master, α ≥ ub + 
                    sum(sum(lambdaN_temp[jj,ss] for ss in 1:ns)*(ysp[jj] - ysp_nonⁱ[jj]) for jj in 1:Δdrto))
                #@info "we are adding this bender optimality cut $(benderscutO)"
                
                # optimality cut was add
                flagO = 1
            
            end # if gap
        else # sum(flag_temp) != nScen
            ###############################
            # --> adding feasibility cuts #
            ###############################
            display(">>>>>> iteration $(ii):F")
            benderscutF = @constraint(master, 0 ≥
                sum(sum(lambdaN_temp[jj,ss] for ss in 1:ns)*(ysp[1,jj] - ysp_nonⁱ[1,jj]) for jj in 1:Δdrto))
            #@info "we are adding this bender feasibility cut $(benderscutF)"
            
            # feasibility cut was add
            flagO = 0
            
        end # if nScen
        
        # checking if number of iterations reached max 
        if ii < benIterMax
            global yspSol_dec = [NaN for kk=1:Δdrto]
            global objFun_dec = NaN
            global optGap_dec = NaN
            global iter_dec = benIterMax
            global time_dec = timeSolBen
            global time_max_dec = timeSolParBen
        end # if < benIterMax
    end # if for benders iterations

    #####################
    # Extracting values #
    #####################
    if isnan(objFun_dec)
        
        outputFun = Dict('f' => 0, 
                         't' => timeSolBen,
                         'ϕ' => objFun_dec,
                         'τ' => time_max_dec, 
                         'i' => iter_dec,
                         'd' => yspSol_dec,
                         'g' => optGap_dec,
                         'o' => optGapArray)
                        
        
        return outputFun
        
    else
        CinArray = Matrix{Float64}(undef,ndrto,ns)
        PArray = Array{Float64}(undef,nfe,ncp,ns)
        CArray = Array{Float64}(undef,nfe,ncp,ns)
        yspArray = Matrix{Float64}(undef,ndrto,ns)

        for ss in 1:ns
            dicTemp = subp(yspSol_dec,C_0,P_0,u_0,xMPC_0,de_0,VmTemp[ss],pScen,solNom)

            # saving files
            CinArray[:,ss] = dicTemp['I']
            PArray[:,:,ss] = dicTemp['P']
            CArray[:,:,ss] = dicTemp['C']
            yspArray[:,ss] = dicTemp['s']
        end

        #calling values of the solved problem    
        outputFun =  Dict('f' => 1,
                    'ϕ' => objFun_dec,
                    't' => time_dec,
                    'τ' => time_max_dec, 
                    'i' => iter_dec,
                    'd' => yspSol_dec,
                    'g' => optGap_dec,
                    'I' => CinArray,
                    'P' => PArray,
                    'C' => CArray,
                    's' => yspArray,
                    'o' => optGapArray)  
        
        return outputFun

    end
end;

<a id="subsec25"></a>

## Plant Model

In [14]:
function bioreactor_ode!(dx, x, p, t)
    C,P = x
    Cin,Vm_plant = p
    
    # Immobilized enzyme or cell catalytic reaction rate (defined by Michaelis-Menten equation)
    rc = Vm_plant*C/(Ks + C)
    
    # reactor 1
    dx[1] = D*(Cin - C) - rc
    dx[2] = rc - D*P
    
    nothing
end;

<a id="sec3"></a>

# Implementing CL-DRTO in closed loop with the plant

Here we run a Monte Carlo Analysis of the CL-DRTO results. The uncertain parameter value is drawn at each MC iteration and the DRTO has access to the lower and upper bound of that distribution to create the scenarios. 

Getting the solution for the nominal scenario

In [15]:
# building MPC's
mpc = MPC_matrices(A,B,C,p,m,q,r);

In [16]:
# Monte Carlo iterations (20)
n_MC = 30

# Simulation time in sampling periods
nsim = 20

# number of scenarios
nS_MC = 3 

# creating uncertain parameter array
pUnc = LinRange(pNom - deltaPar,pNom + deltaPar, nS_MC)

#equiprobable scenarios
pS_MC = 1.0/nS_MC

# Extracting solution for plotting --> monolithic vs. decomposed 
PPlant = Array{Float64}(undef,nsim,n_MC,2)
CPlant = Array{Float64}(undef,nsim,n_MC,2)
CinPlant = Array{Float64}(undef,nsim,n_MC,2)
yspPlant = Array{Float64}(undef,nsim,n_MC,2)
flagCLDRTO = Array{Float64}(undef,nsim,n_MC,2);

## Plant simulation

In [17]:
for jj = 1:n_MC
    # ramdonly drawing the plant from the previously computed models
    d = Truncated(Normal(pNom, deltaPar), pNom - deltaPar, pNom + deltaPar)  #Construct the distribution type
    pPlant = rand(d)
    display("DB: MC iter. $(jj) --> Plant parameter = $(pPlant)")
    
    # RUNNING SIMULATION
    for mm = 1:2
        
        # preparing simulation (starting from opt steady-state)
        # plant 
        xk = x0
        uk_1 = u0

        # mpc's models - initial states is the same
        xk_mpc = x0 - xref
        dek_mpc = 0.0
        
        # initial conditions (CLDRTO)
        Cpk = C0
        Ppk = P0
        
        solNom = MS_CLDRTO(Cpk,Ppk,uk_1,xk_mpc,dek_mpc,pNom,1,0)
        # checking if 1s solution converged
        if solNom['f'] == 1
            display("Nominal solution at t = 0: found")
        else
           display("DB: Nominal solution at t = 0: not found")
        end
        
        for ii = 1:nsim

            print("MC: ", jj)
            println(">> Time: ",ii)

            # updating arrays # States (algebraic and differential): CA1,CB1,CC1,T1,Tj1_out,CA2,CB2,CC2,T2,Tj2_out
            CPlant[ii,jj,mm] = Cpk
            PPlant[ii,jj,mm] = Ppk
            CinPlant[ii,jj,mm] = uk_1

            # executing DRTO
            if ii%Δdrto == 0 || ii == 1
                
                if ii > 1
                    sol_1s_temp = MS_CLDRTO(Cpk,Ppk,uk_1,xk_mpc,dek_mpc,pNom,1,solNom) 
                    # checking if 1s solution converged
                    if sol_1s_temp['f'] == 1
                        display("DB: Nominal solution at t = $(ii): found")
                        solNom = sol_1s_temp 
                    else
                        display("DB: Nominal solution t = $(ii): not found")
                    end
                end
                
                if mm == 1
                    sol_temp = MS_CLDRTO(Cpk,Ppk,uk_1,xk_mpc,dek_mpc,pUnc,nS_MC,solNom)
                else
                    sol_temp = BENS_CLDRTO(Cpk,Ppk,uk_1,xk_mpc,dek_mpc,pUnc,nS_MC,solNom)
                end
                
                # checking solution
                if sol_temp['f'] == 1
                    display("DB: CLDRTO method: $(mm) - Solution at t = $(ii): found")
                    # updating the solution
                    global sol_k = sol_temp
                    # counter for DRTO executions
                    global countDRTO = 1;
                    # N.B. need to declare as global due to loop scope
                else
                    display("DB: CLDRTO method: $(mm) - Solution at t = $(ii): not found")
                    # does not update either the solution or the counter 
                end
            end
            
            # saving flag: surrogate for CLDRTO flag
            flagCLDRTO[ii,jj,mm] = countDRTO

            # Preparing controller 
            # building terms of the OF that depend on current information
            yspk = sol_k['s'][countDRTO:(countDRTO + (p - 1)),2] 
            countDRTO += 1 
            #display("DB: setpoint sent to the controller $(yspk)")
            # N.B.:
            # using nominal (center) scenario, but any scenario could be used

            # solving MPC
            sol_mpc = solveMPC(xk_mpc,dek_mpc,uk_1,yspk,u_mpc_lb,u_mpc_ub,xref[2],uref,mpc,p,m)
            #display("DB: input sent to plant $(sol_mpc['u'][1])")
            
            # evolving plant
            ode_prob = ODEProblem(bioreactor_ode!,xk,tspan,[sol_mpc['u'][1],pPlant]);
            sol_plant = solve(ode_prob,Tsit5(),reltol=1e-8,abstol=1e-8,save_everystep = false); 
            
            # loop
            # -> Plant 
            xk = sol_plant.u[end]
            uk_1 = sol_mpc['u'][1]
            
            # -> for initializing DRTO
            Cpk = xk[1]
            Ppk = xk[2]
            
            # -> MPCs
            # evolving MPC models
            xk_mpc  = A*xk_mpc + B*(uk_1 - uref)
            dek_mpc = Ppk - (xk_mpc[2] + xref[2])
            
            # saving CLDRTO decisions
            yspPlant[ii,jj,mm] = yspk[1]
            
        end # simulation
    end # monolithic || decomposed
end # MC

"DB: MC iter. 1 --> Plant parameter = 0.8682533402406438"


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



"Nominal solution at t = 0: found"

MC: 1>> Time: 1


"DB: CLDRTO method: 1 - Solution at t = 1: found"

...

">>>>>> iteration 3:O"

"DB: opt. gap 0.0010726521709443202"

">>>>>> iteration 4:O"

"DB: opt. gap 0.0003417064446987031"

"DB: CLDRTO method: 2 - Solution at t = 20: found"

Preparing plot

In [18]:
# time series for plotting
ts = Vector{Float64}(undef,nsim) 
for i in 1:nsim
    ts[i] = 1*i
end

# one subplot for each method
num_subplots = 2

# Creating color array
CList = reshape( range(colorant"red", stop=colorant"blue",length=n_MC), 1, n_MC);
MList =[:square, :circle];

Computing statistics

In [19]:
# Extracting solution for plotting --> monolithic vs. decomposed 
CMean = mean(CPlant,dims=2)
PMean = mean(PPlant,dims=2)
CinMean = mean(CinPlant,dims=2)
yspMean = mean(yspPlant,dims=2);

Plotting receipts

In [32]:
@userplot MC_Plot
@recipe function f(h::MC_Plot)
        
    if length(h.args) < 5 
        ylab, x, y_bar, y = h.args
    else
        ylab, x, y_bar, y, lim_u = h.args
    end
    
    # set up the subplots
    # these are common to both marginal histograms
    legend := false
    layout :=  (2,1)
    grid := true
    gridlinewidth --> 2
    latex := true
    linecolor := :black
    
    for mm in 1:2
        # labels 
        ylabel := ylab
        if mm == 1
            title := "Monolithic"
        else
            title := "Decomposed"
            xlabel := "time [min]"
        end
        
        # DATA PLOTTING
        # upper bound
        if length(h.args) > 4 
            # lower bound
            @series begin
                seriestype := :path
                linestyle := :dot
                subplot := mm
                x, lim_u*ones(length(x))
            end
        end

        # average MC value
        @series begin
            seriestype := :path
            linewidth := 3
            
            if mm == 1
                linestyle := :solid
                linecolor := :red
            else
                linestyle := :solid
                linecolor := :black
            end
            
            subplot := mm
            x, y_bar[:,mm]
            
        end
        
        for nn in 1:n_MC
            @series begin
                seriestype := :path
                linewidth := 2
                linealpha := 0.1
                
                if mm == 1
                    linecolor := :red
                else
                    linecolor := :black
                end
                
                subplot := mm
                x, y[:,nn,mm]
            end
        end # MC loop
    end # mono/dec loop 
end

In [21]:
@userplot MC_Comp
@recipe function f(h::MC_Comp)
        
    if length(h.args) < 4 
        ylab, x, y_bar = h.args
    else
        ylab, x, y_bar, lim_u = h.args
    end
    
    # set up the subplots
    # these are common to both marginal histograms    
    grid := true
    gridlinewidth --> 2
    
    linecolor := :black
    markercolor := :black
    markerstrokecolor := :black
    markersize := 3
    
    # axis labels 
    ylabel := ylab
    xlabel := "time [min]"

    # legend
    legend := true
    latex := true
    labels = ["Mono.","Decom."] 
    
    # DATA PLOTTING
    # upper bound
    if length(h.args) > 3
        # lower bound
        @series begin
            seriestype := :path
            linestyle := :dot
            label --> ""
            x, lim_u*ones(length(x))
        end
    end

    # monolithic
    @series begin
        seriestype := :path
        linewidth := 3
        linestyle := :solid
        linecolor := :red
        label --> "Mono."
        x, y_bar[:,1]
    end 
    
    # decomposed
    @series begin
        seriestype := :path
        linewidth := 3
        linestyle := :solid
        linecolor := :black
        label --> "Deco."
        x, y_bar[:,2]
    end 
end

## Monte Carlo Iterations 

- Biomass

In [33]:
mc_plot(L"P [g/l]",ts,PMean[:,1,:],PPlant)

savefig("C:/Users/MACC-Jose/Documents/Canada/Papers/Benders Decomposition/Figures/C4_MC_outputs.pdf");

- Biomass Setpoint

In [34]:
mc_plot(L"P^{SP} [g/L]",ts,yspMean[:,1,:],yspPlant,yspMax)

savefig("C:/Users/MACC-Jose/Documents/Canada/Papers/Benders Decomposition/Figures/C4_MC_setpoints.pdf");

- Inputs

In [35]:
mc_plot(L"C_{in} [g/L]",ts,CinMean[:,1,:],CinPlant,Cin_max)

savefig("C:/Users/MACC-Jose/Documents/Canada/Papers/Benders Decomposition/Figures/C4_MC_inputs.pdf");

## Comparing Average Behavior

- Biomass

In [25]:
mc_comp(L"P [g/L]",ts,PMean[:,1,:])

savefig("C:/Users/MACC-Jose/Documents/Canada/Papers/Benders Decomposition/Figures/C4_MC_ave_outputs.pdf");

- Biomass Setpoint

In [26]:
mc_comp(L"P^{SP} [g/L]",ts,yspMean[:,1,:],yspMax)

savefig("C:/Users/MACC-Jose/Documents/Canada/Papers/Benders Decomposition/Figures/C4_MC_ave_setpoints.pdf");

- Inputs

In [27]:
mc_comp(L"C_{in} [g/L]",ts,CinMean[:,1,:],Cin_max)

savefig("C:/Users/MACC-Jose/Documents/Canada/Papers/Benders Decomposition/Figures/C4_MC_ave_inputs.pdf");